In [3]:
import numpy as np
import pandas as pd
import stanza

# stanza.download('he', processors='tokenize,pos,lemma,depparse')
nlp = stanza.Pipeline('he', processors='tokenize,pos,lemma,depparse')



2025-05-07 18:06:06 WARNING: Language he package default expects mwt, which has been added
2025-05-07 18:06:06 INFO: Loading these models for language: he (Hebrew):
| Processor | Package |
-----------------------
| tokenize  | htb     |
| mwt       | htb     |
| pos       | htb     |
| lemma     | htb     |
| depparse  | htb     |

2025-05-07 18:06:06 INFO: Use device: cpu
2025-05-07 18:06:06 INFO: Loading: tokenize
/Users/pabloherrero/Library/Python/3.9/lib/python/site-packages/stanza/models/tokenization/trainer.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be exe

# Test package

In [4]:
doc = nlp('הדר לא מותצה כלום להסתכל בטלוויזיה')
for sent in doc.sentences:
    for word in sent.words:
        print(word.text, word.lemma, word.upos, word.head, word.feats, word.deprel)

ה ה DET 2 Definite=Def|PronType=Art det
דר דר NOUN 4 Gender=Masc|Number=Sing nsubj
לא לא ADV 4 Polarity=Neg advmod
מותצה הותצה VERB 0 Gender=Masc|HebBinyan=HUFAL|Number=Sing|Person=1,2,3|VerbForm=Part|Voice=Pass root
כלום כלום ADV 4 None advmod
להסתכל הסתכל VERB 4 HebBinyan=HITPAEL|VerbForm=Inf xcomp
בטלוויזיה בטלוויזיה NOUN 6 Gender=Fem|Number=Sing obj


# Test on database

In [5]:
lippath = '/Users/pabloherrero/Documents/ManHatTan/data/processed/LIPSTICK/hebrew_db.lip'
lip = pd.read_csv(lippath)
tokens_list = list(lip['word_ll'].values)
tokens_list[:6]

['מלגה', 'כונה', 'מגבת', 'להגיע', 'ציון', 'מסך']

In [6]:
tokens_list = list(lip['word_ll'].values)

lexemes = []
for token in tokens_list:
    doc = nlp(token)
    # lernt/lernen<vblex><pri><p3><sg>,
    for sent in doc.sentences:
        lexeme_string = ''
        # print(len(sent.words))
        for word in sent.words:
            lexeme_string += word.text 
            if word.lemma:
                lexeme_string += '/' + word.lemma 
            if word.upos:
                lexeme_string += '<' + word.upos + '>'
            if word.feats:
                feats = word.feats.split('|')
                lexeme_string += ''.join(feat.split('=')[1] +'|' for feat in feats)
            if word.deprel != 'root':
                lexeme_string +=  ',' + word.deprel
        lexemes.append(lexeme_string)


In [7]:
# lexemes
lip.loc[:, 'lexeme_string'] = lexemes

In [8]:
lip.tail(8)

,p_recall,n_id,timestamp,delta,user_id,learning_language,ui_language,word_ll,word_ul,lexeme_string,...,mdt_history,mdt_correct,mrt_history,mrt_correct,wdt_history,wdt_correct,wrt_history,wrt_correct,speed,rebag
285,0.0,557,1746629700,0,pablo,iw,en,בעיקר,Mainly,בעיקר/בעיקר<ADV>,...,0,0,0,0,0,0,0,0,0.0,NaN
286,0.0,442,1746629700,0,pablo,iw,en,בשבילך,for you,"בשביל_/בשביל<ADP>,case_אתה/הוא<PRON>Masc|Sing|...",...,0,0,0,0,0,0,0,0,0.0,NaN
287,0.0,280,1746629700,0,pablo,iw,en,נרגילה,hookah,נרגילה/נרגילה<NOUN>Fem|Sing|,...,0,0,0,0,0,0,0,0,0.0,NaN
288,0.0,90,1746629700,0,pablo,iw,en,צבא,Army,צבא/צבא<NOUN>Masc|Sing|,...,0,0,0,0,0,0,0,0,0.0,NaN
289,0.0,572,1746629700,0,pablo,iw,en,חמוד,Cute,חמוד/חמוד<ADJ>Masc|Sing|,...,0,0,0,0,0,0,0,0,0.0,NaN
290,0.0,163,1746629700,0,pablo,iw,en,ביקשת,you asked,ביקשת/ביקש<VERB>Masc|PIEL|Sing|1|Past|Act|,...,0,0,0,0,0,0,0,0,0.0,NaN
291,0.0,150,1746629700,0,pablo,iw,en,להתקלח,shower,להתקלח/התקלח<VERB>HITPAEL|Inf|,...,0,0,0,0,0,0,0,0,0.0,NaN
292,0.0,561,1746629700,0,pablo,iw,en,חצוף,insolent,חצוף/חצוף<ADJ>Masc|Sing|,...,0,0,0,0,0,0,0,0,0.0,NaN


## Write function:

In [145]:
def get_lexemes(lip):
    tokens_list = list(lip['word_ll'].values)

    lexemes = []
    for token in tokens_list:
        doc = nlp(token)
        # lernt/lernen<vblex><pri><p3><sg>,
        for sent in doc.sentences:
            lexeme_string = ''
            # print(len(sent.words))
            for word in sent.words:
                lexeme_string += word.text 
                if word.lemma:
                    lexeme_string += '/' + word.lemma 
                if word.upos:
                    lexeme_string += '<' + word.upos + '>'
                if word.feats:
                    feats = word.feats.split('|')
                    lexeme_string += ''.join(feat.split('=')[1] +'|' for feat in feats)
                if word.deprel != 'root':
                    lexeme_string +=  ',' + word.deprel
            lexemes.append(lexeme_string)
    return lexemes

In [146]:
def test_get_lexemes(lip):
    lexemes = get_lexemes(lip)
    assert len(lexemes) == len(lip)

In [9]:
lip.to_csv(lippath, index=False)

In [10]:
lip

,p_recall,n_id,timestamp,delta,user_id,learning_language,ui_language,word_ll,word_ul,lexeme_string,...,mdt_history,mdt_correct,mrt_history,mrt_correct,wdt_history,wdt_correct,wrt_history,wrt_correct,speed,rebag
0,0.714,190,1743319778,0,pablo,iw,en,מלגה,scholarship,מלגה/מלגה<NOUN>Fem|Sing|,...,5,5,0,0,2,0,0,0,0.1459,True
1,0.857,98,1744291702,971924,pablo,iw,en,כונה,intention,כונה/כונה<VERB>Masc|PUAL|Sing|3|Past|Pass|,...,4,4,0,0,2,2,0,0,0.0835,True
2,1.000,501,1744182216,862438,pablo,iw,en,מגבת,towel,"מגבת/גיבת<VERB>Fem|PAAL|Sing|1,2,3|Part|Act|",...,2,2,0,0,4,4,0,0,0.1115,True
3,1.000,280,1746598161,3278383,pablo,iw,en,להגיע,to arrive,להגיע/הגיע<VERB>HIFIL|Inf|Act|,...,2,2,0,0,4,4,0,0,0.0713,False
4,1.000,390,1744122447,802669,pablo,iw,en,ציון,Grade,ציון/ציון<PROPN>,...,6,6,0,0,0,0,0,0,0.1102,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,0.000,90,1746629700,0,pablo,iw,en,צבא,Army,צבא/צבא<NOUN>Masc|Sing|,...,0,0,0,0,0,0,0,0,0.0000,NaN
289,0.000,572,1746629700,0,pablo,iw,en,חמוד,Cute,חמוד/חמוד<ADJ>Masc|Sing|,...,0,0,0,0,0,0,0,0,0.0000,NaN
290,0.000,163,1746629700,0,pablo,iw,en,ביקשת,you asked,ביקשת/ביקש<VERB>Masc|PIEL|Sing|1|Past|Act|,...,0,0,0,0,0,0,0,0,0.0000,NaN
291,0.000,150,1746629700,0,pablo,iw,en,להתקלח,shower,להתקלח/התקלח<VERB>HITPAEL|Inf|,...,0,0,0,0,0,0,0,0,0.0000,NaN
